In [32]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [6]:
url = 'https://www.tranzita.com'

In [7]:
model = Ollama(model='llama3.1')

In [10]:
loader = WebBaseLoader(url)
document = loader.load()

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
final_doc = text_splitter.split_documents(document)

In [15]:
len(final_doc)

3

In [19]:
final_doc[2].page_content

'Microsoft Power BI\nVisualize your data with Microsoft Power BI\n\n\n\n\n\n\n\n\n\n\n\n\n\nCyber Security Audit\nSecure your applications and severs through VA/PT audit\n\n\n\n\n\n\n\n\n\n\n\n\n\nMobile First\nAll your software needs at your finger tips.\n\n\n\n\n\n\n\n\n\n\n\n\n\nModern Web\nHigh performance web application for rich UX.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJoin us in building the best in class products with cutting edge technologies.\n\n\n\n\n\n\n\n\n\n\nContact us\nsubmit your query and we will drop you an email.\n\n\n\n\n\nWrite your email *\n\n\n\n\nMessage *\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n© 2021-22 Tranzita Systems. Designed with  by Tranzita Systems.'

In [22]:
ollama_embeddings = OllamaEmbeddings(
    model='llama3.1',
    model_kwargs={'device':'gpu'}
)

In [25]:
vectors = FAISS.from_documents(final_doc,ollama_embeddings)

In [34]:
retriever  = vectors.as_retriever()

In [24]:
# pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 1.5 MB/s eta 0:00:0000:0100:02
Note: you may need to restart the kernel to use updated packages.


In [30]:
prompt = ChatPromptTemplate.from_template(
"""
Provide me the accurate answers regarding the context.
<context>
{context}
<context>
Question:{input}
"""
)


In [31]:
document_chain = create_stuff_documents_chain(model,prompt)

In [35]:
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [36]:
question = 'tell me in brief about the company.'

In [39]:
response = retrieval_chain.invoke({'input':question})

In [41]:
response['answer']

'Based on the provided context, here\'s a brief overview of Tranzita Systems:\n\nTranzita Systems is a company that provides various services and solutions related to data management, analytics, and technology. They specialize in areas such as:\n\n* Data Engineering: Transforming data across cloud platforms\n* Data Science: Utilizing data for defects detection and predictive maintenance\n* BI Dashboard: Visualizing data and gaining key insights\n* PLC Programming: Automating tasks through robotics\n\nTheir expertise extends to IoT, Machine Learning, and Big Data. The company also emphasizes the importance of Cyber Security Audit to secure applications and servers.\n\nThe tone of the website suggests a forward-thinking approach, with phrases like "Join us in building the best-in-class products" and "Transform your data into information." Overall, Tranzita Systems appears to be a technology-driven organization focused on helping businesses solve complex problems through innovative soluti